In [4]:
import pandas as pd
import numpy as np
import pandas as pd
import re
from nltk.tokenize import sent_tokenize

In [5]:
test = pd.read_json("test.json","r", encoding="utf8")
train = pd.read_json("train.json","r", encoding="utf8")

In [6]:
df = pd.concat([test,train])

In [7]:
df.head()

,NNQT_question,uid,subgraph,template_index,question,sparql_wikidata,sparql_dbpedia18,template,answer,template_id,paraphrased_question
0,What is the {country} for {head of state} of {...,20258,simple question left,604,Who is the {country} for {head of state} of {...,select distinct ?sbj where { ?sbj wdt:P35 wd:...,select distinct ?subj where { ?statement <http...,<?S P O ; ?S InstanceOf Type>,[],2,What country is Mahmoud Abbas the head of stat...
1,What is {population} of {Somalia} that is {poi...,7141,statement_property,3586,What was the population of Somalia in 2009-0-0?,SELECT ?obj WHERE { wd:Q1045 p:P1082 ?s . ?s p...,select distinct ?obj where {\n?statement <htt...,(E pred F) prop ?value,[],statement_property_2,"As of 2009, how many people lived in Somalia?"
2,"What is {voice actresses} of {South Park}, tha...",12761,right-subgraph,5331,Which female actress is the voice over on Sout...,SELECT ?answer WHERE { wd:Q16538 wdt:P725 ?ans...,SELECT ?answer WHERE { ?statement1 <http://www...,E REF ?F . ?F RFG G,[],1,Which female actress on South Park is the voic...
3,What is {nominated for} of {Dolores del Río} t...,4236,statement_property,681,What is {nominated for} of {Dolores del Río} t...,SELECT ?obj WHERE { wd:Q124057 p:P1411 ?s . ?s...,select distinct ?obj where {\n?statement <htt...,(E pred F) prop ?value,[],statement_property_2,What nomination did Dolores del Rio receive fo...
4,What is {population} of {Clermont-Ferrand} tha...,8669,statement_property,5114,What was the population of Clermont-Ferrand on...,SELECT ?obj WHERE { wd:Q42168 p:P1082 ?s . ?s ...,select distinct ?obj where {\n?statement <htt...,(E pred F) prop ?value,[],statement_property_2,How many people lived in Clermont-Ferrand on J...


In [14]:
sentences = []
for x in df.paraphrased_question.apply(str):
    if x == '[]':
        continue
    sentences.append(x)

In [15]:
sentences

['What country is Mahmoud Abbas the head of state of?',
 'As of 2009, how many people lived in Somalia?',
 'Which female actress on South Park is the voice over and is used as a singer?',
 'What nomination did Dolores del Rio receive for their work with La Otra?',
 'How many people lived in Clermont-Ferrand on January 1st, 2013?',
 'In what open cluster is the radius bigger?',
 'When did Jonathan Swift, a resident of Whitehaven, die?',
 'Reveal to ME MODE OF TRANSPORT WHOSE NAME HAS THE WORD VEHICLE IN IT.',
 'Which score got Danny Elfman nominated for an Academy Award for Best Original Score?',
 'What is the title of the musical drama based on Twelfth Night ?',
 'Which statement of subject was Franz Waxman nominated for at the 23rd Academy Awards?',
 'In which region does the Rideau Canal join the Ottawa River?',
 'At the Academy Awards, which actor was nominated for their work in the Truman Show for Best Supporting Actor?',
 'what are the cinematic strategy that contains the word til

In [16]:
from sentence_transformers import SentenceTransformer, util

embedder = SentenceTransformer('bert-base-nli-mean-tokens')

corpus_embeddings = embedder.encode(sentences, convert_to_tensor=True)

In [17]:
closest_n = 5
queries = ['when the first world war ended',"some juventus players"]

for query in queries:
    query_embedding = embedder.encode(query, convert_to_tensor=True)
    scores = util.pytorch_cos_sim(query_embedding, corpus_embeddings)[0]

    results = zip(range(len(scores)), scores)
    results = sorted(results, key=lambda x: x[1], reverse=True)

    print("\n\n======================\n\n")
    print("Query:", query)
    print("\nTop 5 most similar sentences in corpus:")

    for idx, score in results[0:closest_n]:
        print(sentences[idx].strip(), "(Score: %.4f)" % (score))





Query: when the first world war ended

Top 5 most similar sentences in corpus:
The year King Henry I monarch reign ended? (Score: 0.6609)
There is a country in Beersheba that has am end time of 1917-10-31, which one is it? (Score: 0.6498)
When was the position of Alfred, Ruler Tennyson as Writer Laureate of the Joined together Kingdom came to an end? (Score: 0.6400)
What is the forerunner of the contributing variables to World War I? (Score: 0.6356)
What head of state starts from The Moment World War? (Score: 0.6347)




Query: some juventus players

Top 5 most similar sentences in corpus:
The Juventus F.C. has what number of team participants? (Score: 0.7259)
Which is the number of sponsors that are for Juventus F.C.? (Score: 0.7066)
Give me the year that Atillo Ubertalli's chairman was Juventus F.C. (Score: 0.6723)
Which is the number of things that Juventus F.C. found? (Score: 0.6211)
how much number of matches played for Giuseppe Meazza contains a part of sports group as Atalan